# Import

In [1]:
import tensorflow as tf
import argparse

import numpy as np
import pandas as pd

/Users/meihuaren/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
all_features = pd.read_csv('all_features_new_64.csv')

# After PCA or AutoEncoder，features do not contain 'ask_price_1' and 'bid_price_1', 
# thus, we calculate and store the label first though there is no use in the feature selection part.
data = all_features.fillna(method='ffill')
data['mid_price'] = (data['ask_price_1'] + data['bid_price_1']) / 2
data['d_price'] = data['mid_price'].diff().shift(-1)
data['label'] = 1*(data['d_price']>0) - 1*(data['d_price']<0)
data = data.dropna() # drop the first 6 rows (with some nan features) and the last row (with nan 'd_price')
data = data.drop(['mid_price', 'd_price'], axis=1)

data = data.reset_index()
data = data.drop(['index'], axis=1)

In [3]:
print(data.shape)
data.head()

(581023, 65)


,ask_price_1,ask_vol_1,bid_price_1,bid_vol_1,ask_price_2,ask_vol_2,bid_price_2,bid_vol_2,ask_price_3,ask_vol_3,...,rank_bid_vol_4,rank_ask_vol_4,rank_bid_vol_5,rank_ask_vol_5,corr_vol_1,corr_vol_2,corr_vol_3,corr_vol_4,corr_vol_5,label
0,275200,166,275100,300,275300,1000,275000,100,275400,373,...,1.0,1.0,0.714286,1.0,-0.353553,-1.0,-1.0,-1.0,1.0,0
1,275200,166,275100,300,275300,1000,275000,100,275400,373,...,1.0,1.0,0.750000,1.0,-0.377964,-1.0,-1.0,-1.0,1.0,0
2,275200,166,275100,300,275300,1000,275000,100,275400,373,...,1.0,1.0,1.000000,1.0,-0.395285,-1.0,-1.0,-1.0,1.0,0
3,275200,166,275100,300,275300,1000,275000,300,275400,373,...,1.0,1.0,1.000000,1.0,-0.408248,-1.0,-1.0,-1.0,1.0,0
4,275200,100,275100,300,275300,1000,275000,300,275400,373,...,1.0,1.0,1.000000,1.0,-0.228210,1.0,-1.0,1.0,1.0,1


In [4]:
train_weight = 0.8
split = int(data.shape[0] * train_weight)
df_train = data.iloc[:split,:-1]
df_test = data.iloc[split:,:-1]

nrow = 3000
df_valid = df_test[0:nrow]
df_test = df_test[nrow:]

x_train = df_train.values
x_valid = df_valid.values
x_test = df_test.values
x_all = data.iloc[:,:-1].values

In [5]:
#normalization (to make sure the autoencoder is converging)
x_max = np.max(x_train,axis=0)
x_min = np.min(x_train,axis=0)
x_train = (x_train - x_min) / (x_max - x_min)
x_valid = (x_valid - x_min) / (x_max - x_min)
x_test = (x_test - x_min) / (x_max - x_min)
x_all = (x_all - x_min) / (x_max - x_min)

In [6]:
print(x_train.shape, x_valid.shape, x_test.shape, x_all.shape)

(464818, 64) (3000, 64) (113205, 64) (581023, 64)


# Definition of the Architecture

In [7]:
input_size = 64
#re-constructed size
output_size = 64

# 3 hidden layers for encoder
n_encoder_h_1 = 32
n_encoder_h_2 = 16
n_encoder_h_3 = 8

# 3 hidden layers for decoder
n_decoder_h_1 = 8
n_decoder_h_2 = 16
n_decoder_h_3 = 32

# Parameters
learning_rate = 0.01
training_epochs = 200 #200
batch_size = 200
display_step = 1

# Batch Normalization 

In [8]:
def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')

    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
       
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    #normed = tf.nn.batch_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])


# Definition of the Layer 

In [9]:
def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))
    # Using sigmoid to avoid sharp transitions in neurons

# Definition of the Encoder Part

In [10]:
def encoder(x, n_code, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder"):
        
        with tf.variable_scope("h_1"):
            h_1 = layer(x, [input_size, n_encoder_h_1], [n_encoder_h_1], phase_train)

        with tf.variable_scope("h_2"):
            h_2 = layer(h_1, [n_encoder_h_1, n_encoder_h_2], [n_encoder_h_2], phase_train)

        with tf.variable_scope("h_3"):
            h_3 = layer(h_2, [n_encoder_h_2, n_encoder_h_3], [n_encoder_h_3], phase_train)

        with tf.variable_scope("code"):
            output = layer(h_3, [n_encoder_h_3, n_code], [n_code], phase_train)

    return output

# Definition of the Decoder Part

In [11]:
def decoder(x, n_code, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    
    with tf.variable_scope("decoder"):
        
        with tf.variable_scope("h_1"):
            h_1 = layer(x, [n_code, n_decoder_h_1], [n_decoder_h_1], phase_train)

        with tf.variable_scope("h_2"):
            h_2 = layer(h_1, [n_decoder_h_1, n_decoder_h_2], [n_decoder_h_2], phase_train)

        with tf.variable_scope("h_3"):
            h_3 = layer(h_2, [n_decoder_h_2, n_decoder_h_3], [n_decoder_h_3], phase_train)

        with tf.variable_scope("output"):
            output = layer(h_3, [n_decoder_h_3, output_size], [output_size], phase_train)

    return output

# Definition of the Loss

In [12]:
# loss is L2 measure
def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    with tf.variable_scope("training"):
        
        l2_measure = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2_measure)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op

# Training Function

In [13]:
# Using Adam as optimizer for training 	
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

# Evaluation Function

In [14]:
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        - output: prediction vector of the network for the validation set
        - x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        return val_loss, val_summary_op

# Main Function - Training

In [15]:
if __name__ == '__main__':

    #parser = argparse.ArgumentParser(description='Autoencoder')
    #parser.add_argument('n_code', nargs=1, type=str)
    #args = parser.parse_args(['--help'])
    #n_code = args.n_code[0]
    
    #if a jupyter file, please comment the 4 above and use the one bellow
    n_code = '8'
    
    #feel free to change with your own
    model_path = '/Users/meihuaren/personal/DL_logs/ae64/'

    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):


            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            x = tf.placeholder("float", [None, 64]) # 64 original features
            
            phase_train = tf.placeholder(tf.bool)

            #define the encoder 
            code = encoder(x, int(n_code), phase_train)

            #define the decoder
            output = decoder(code, int(n_code), phase_train)

            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            eval_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            saver = tf.train.Saver()

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            train_writer = tf.summary.FileWriter(model_path, graph=sess.graph)
            val_writer   = tf.summary.FileWriter(model_path, graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)

            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                total_batch = int(x_train.shape[0]/batch_size)
                
                #train_writer = tf.summary.FileWriter(model_path+str(epoch)+'/model.ckpt', graph=sess.graph)
                #val_writer   = tf.summary.FileWriter(model_path+str(epoch)+'/model.ckpt', graph=sess.graph)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    minibatch_x = x_train[i*batch_size:(i+1)*batch_size]
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:.9f}".format(avg_cost))

                    train_writer.add_summary(train_summary, sess.run(global_step))

                    validation_loss, val_summary = sess.run([eval_op, val_summary_op], feed_dict={x: x_valid, phase_train: False})
                    
                    val_writer.add_summary(val_summary, sess.run(global_step))
                    
                    print("Validation Loss:", validation_loss)

                    save_path = saver.save(sess, model_path)
                    print("Model saved in file: %s" % save_path)


            print("Optimization Done")

            test_loss = sess.run(eval_op, feed_dict={x: x_test, phase_train: False})

            print("Test Loss:", test_loss)

Epoch: 0001 cost = 1.828693544
Validation Loss: 1.4115511
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0002 cost = 1.471254427
Validation Loss: 1.3648889
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0003 cost = 1.417240907
Validation Loss: 1.3217626
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0004 cost = 1.395498292
Validation Loss: 1.3179958
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0005 cost = 1.384724677
Validation Loss: 1.3060199
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0006 cost = 1.376183936
Validation Loss: 1.3200765
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0007 cost = 1.368336022
Validation Loss: 1.3127815
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0008 cost = 1.360768943
Validation Loss: 1.3219726
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0009 cost = 1.352533446
Validation Loss: 

Epoch: 0070 cost = 1.237225896
Validation Loss: 1.5705512
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0071 cost = 1.236384526
Validation Loss: 1.5321953
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0072 cost = 1.235589726
Validation Loss: 1.6052591
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0073 cost = 1.234861188
Validation Loss: 1.5729223
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0074 cost = 1.234262866
Validation Loss: 1.6541401
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0075 cost = 1.233669990
Validation Loss: 1.477049
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0076 cost = 1.233243267
Validation Loss: 1.5816472
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0077 cost = 1.232564915
Validation Loss: 1.6875232
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0078 cost = 1.232067423
Validation Loss: 1

Epoch: 0139 cost = 1.208416206
Validation Loss: 1.9005349
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0140 cost = 1.208116758
Validation Loss: 2.0654821
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0141 cost = 1.207921620
Validation Loss: 1.9039054
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0142 cost = 1.207538313
Validation Loss: 1.7699167
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0143 cost = 1.207428392
Validation Loss: 1.7056386
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0144 cost = 1.207130727
Validation Loss: 1.8719817
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0145 cost = 1.206993570
Validation Loss: 1.8541175
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0146 cost = 1.206699217
Validation Loss: 1.6692148
Model saved in file: /Users/meihuaren/personal/DL_logs/ae64/
Epoch: 0147 cost = 1.206497582
Validation Loss: 

# Main Function - Extracting Features

In [16]:
if __name__ == '__main__':

    #feel free to change with your own
    args_savepath = '/Users/meihuaren/personal/DL_logs/ae64/'
    new_features_resultpath = '/Users/meihuaren/personal/OR_2018fall/Courses/E4720 Deep Learning/project_coding/Team E_code/'
    n_code = 8

    #=====================================
    # AutoEncoder

    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            x = tf.placeholder("float", [None, 64]) # 64 original features
            
            phase_train = tf.placeholder(tf.bool)

            code = encoder(x, n_code, phase_train)

            output = decoder(code, n_code, phase_train)

            cost, train_summary_op = loss(output, x)

            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            eval_op, val_summary_op = evaluate(output, x)

            #saver = tf.train.Saver()
            #sess = tf.Session()
            print('\n')
            print('Starting Autoencoder', args_savepath ) #args.savepath[0]
            print('\n')
            
            sess = tf.Session()
            saver = tf.train.Saver()
            save_path = saver.restore(sess, args_savepath ) #args.savepath[0]
            print("Model restored from file: %s" % save_path)

            print('Running Autoencoder & Autoencoder Codes')
            print('\n')
            
            ae_codes = sess.run(code, feed_dict={x: x_all, phase_train: True})
            print(ae_codes)
            ae_codes_df = pd.DataFrame(ae_codes)
            features64_new_ae = pd.concat([ae_codes_df,data.iloc[:,-1]],axis = 1)
            filename = new_features_resultpath + 'features64_new_ae.csv'
            features64_new_ae.to_csv(filename, index=False)
            
            #ae_codes, ae_reconstruction = sess.run([code, output], feed_dict={x: mnist.test.images*np.random.randint(2, size=(784)), phase_train: True})
            



Starting Autoencoder /Users/meihuaren/personal/DL_logs/ae64/


INFO:tensorflow:Restoring parameters from /Users/meihuaren/personal/DL_logs/ae64/
Model restored from file: None
Running Autoencoder & Autoencoder Codes


[[0.88007325 0.19270101 0.01311367 ... 0.09857836 0.9316895  0.16101304]
 [0.88009614 0.19244654 0.01317205 ... 0.09935102 0.9315308  0.16085123]
 [0.8801059  0.19107047 0.01321665 ... 0.09986044 0.93139416 0.16337244]
 ...
 [0.8865644  0.20159969 0.00806681 ... 0.09982315 0.9387753  0.16453943]
 [0.8866113  0.20136005 0.0080296  ... 0.09983772 0.9412486  0.16444616]
 [0.886813   0.20118488 0.00806309 ... 0.0996583  0.94127446 0.16400476]]
